In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [3]:
import sys
PROJECT_PATH = '/content/drive/My Drive/END/'
sys.path.append(PROJECT_PATH +'/Session5/')
import os
os.chdir(PROJECT_PATH +'/Session5/')

In [4]:
import pandas as pd

data = pd.read_csv("datasetSentences.csv",encoding = "ISO-8859-1", engine='python')
label = pd.read_csv("sentiment_labels.csv",encoding = "ISO-8859-1", engine='python')
data['labels'] = label[['score']]
data.head()
df = data.copy()

In [5]:
print(df.shape)
df.head()
df.Sentence[0]

(11855, 2)


"The Rock is destined to be the 21st Century 's new `` Conan '' and that he 's going to make a splash even greater than Arnold Schwarzenegger , Jean-Claud  Van Damme or Steven Segal ."

In [6]:
df.labels.value_counts()

2    6240
1    2175
3    2139
4     663
0     638
Name: labels, dtype: int64

In [7]:
import random
import torch, torchtext
from torchtext import data

In [8]:
# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [9]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [10]:
!pip install googletrans==3.1.0a0

     |████████████████████████████████| 61kB 4.6MB/s 
     |████████████████████████████████| 1.0MB 15.2MB/s 
     |████████████████████████████████| 51kB 7.9MB/s 
     |████████████████████████████████| 71kB 11.7MB/s 
     |████████████████████████████████| 61kB 8.6MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=ae4ed35802346afe959d0873e7bad6d46628849e29378b2119c5cf9aeac955fa
  Stored in directory: /root/.cache/pip/wheels/27/7a/a0/aff3babbb775549ce6813cb8fa7ff3c0848c4dc62c20f8fdac
Successfully built googletrans


In [11]:
import random
import googletrans
from googletrans import Translator
#import googletrans.Translator
def translation(sentence):
  translator = Translator()
  available_langs = list(googletrans.LANGUAGES.keys()) 
  trans_lang = random.choice(available_langs) 
  #print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

  
  try:
    translations = translator.translate(sentence, dest=trans_lang)
    t_text = [t.text for t in translations]
    translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
    en_text = [t.text for t in translations_en_random]
  except:
    en_text = sentence


  #print(t_text)

  
  return en_text


In [12]:
#df['tran_sentence'] = df.apply(lambda x:translation(x['Sentence']), axis =1)
#df.head()

In [13]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from random import randrange

def remove_stopwords_swap(example_sent):
  stop_words = set(stopwords.words('english'))
  myset = {".", "-", "...", "s", "'"}

  word_tokens = word_tokenize(example_sent)
  filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
  filtered_sentence = []
  rndpos = randrange(len(word_tokens)-1)
  counter = 0
  
  for w in word_tokens:
    
    if (w not in stop_words) and (w not in myset):
      if counter == rndpos:
        syns = wordnet.synsets(w)
        try:
          if syns[0].lemmas()[0].name() is not None:
            rw = syns[0].lemmas()[0].name()
            #print(w,rw)
            filtered_sentence.append(rw)
          else:
              filtered_sentence.append(w)
        except:
            pass


      else:
        filtered_sentence.append(w)
        #print(counter, rndpos)
        #print(filtered_sentence)
    counter = counter +1   
    
    final = ' '.join(filtered_sentence).split(', ')
    #final = [[' '.join(i)] for i in filtered_sentence]
  
  return final[0]



In [14]:


df['new_sentence'] = df.apply(lambda x:remove_stopwords_swap(x['Sentence']), axis =1)
df['length'] = df['new_sentence'].apply(len)
dfn = df[df['length']!=0].reset_index().copy()



In [15]:
(dfn.head())
print(dfn.shape)
print(max(dfn.index))

(11839, 5)
11838


In [16]:
Tweet = torchtext.legacy.data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = torchtext.legacy.data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

In [17]:
type(Tweet)

torchtext.legacy.data.field.Field

In [18]:
fields = [('tweet', Tweet), ('label', Label)]

In [19]:
example = [torchtext.legacy.data.Example.fromlist([dfn.new_sentence[i],dfn.labels[i]], fields) for i in range(dfn.shape[0])] 

In [20]:
type(example)
print(*example)

<torchtext.legacy.data.example.Example object at 0x7f010fbc7550> <torchtext.legacy.data.example.Example object at 0x7f010fbc7690> <torchtext.legacy.data.example.Example object at 0x7f010fbc7650> <torchtext.legacy.data.example.Example object at 0x7f010fbc7610> <torchtext.legacy.data.example.Example object at 0x7f0110bf2c10> <torchtext.legacy.data.example.Example object at 0x7f010ec26c50> <torchtext.legacy.data.example.Example object at 0x7f010ec26d90> <torchtext.legacy.data.example.Example object at 0x7f010ec26fd0> <torchtext.legacy.data.example.Example object at 0x7f010ec1fb90> <torchtext.legacy.data.example.Example object at 0x7f010ec26b90> <torchtext.legacy.data.example.Example object at 0x7f010ec38350> <torchtext.legacy.data.example.Example object at 0x7f010ec38490> <torchtext.legacy.data.example.Example object at 0x7f010ec38810> <torchtext.legacy.data.example.Example object at 0x7f010ec38b10> <torchtext.legacy.data.example.Example object at 0x7f010ec38e50> <torchtext.legacy.data.ex

In [21]:
twitterDataset = torchtext.legacy.data.Dataset(example, fields)

In [22]:
(train, valid) = twitterDataset.split(split_ratio=[85, 15], random_state = random.seed(SEED))

In [23]:
len(train), len(valid)

(10063, 1776)

In [24]:
print(type(valid))
print((vars(valid.examples[1]))['tweet'])
#' '.join(filtered_sentence).split(', ')
for i in range(1,10):
  print(' '.join(vars(valid.examples[i])['tweet']).split(', ')[0])

<class 'torchtext.legacy.data.dataset.Dataset'>
['A', 'pacify', 'engrossing', 'character', 'study']
A pacify engrossing character study
A compelling yarn
however stale material
The movie worked right final scene
Boisterous
A reminder beyond hype recent digital glitz
An unfortunate title film nothing endearing
World_Health_Organization movie ?
Vulgar optimistic title


In [25]:
Tweet.build_vocab(train)
Label.build_vocab(train)

In [26]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)

Size of input vocab :  14835
Size of label vocab :  5
Top 10 words appreared repeatedly : [("'s", 1883), ('-', 1831), ('The', 1266), ('A', 937), ('movie', 889), ('film', 865), ('It', 745), ('`', 565), ("n't", 480), ('?', 396)]
Labels :  defaultdict(None, {2: 0, 1: 1, 3: 2, 4: 3, 0: 4})


In [27]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [28]:
train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweet),
                                                            sort_within_batch=True, device = device)

In [29]:
next(iter(train_iterator))



[torchtext.legacy.data.batch.Batch of size 32]
	[.tweet]:('[torch.cuda.LongTensor of size 32x3 (GPU 0)]', '[torch.cuda.LongTensor of size 32 (GPU 0)]')
	[.label]:[torch.cuda.LongTensor of size 32 (GPU 0)]

In [30]:
import os, pickle
with open('tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

In [31]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [32]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)
embedding_dim = 300
num_hidden_nodes = 100
num_output_nodes = 5
num_layers = 2
dropout = 0.2

# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [33]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(14835, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=5, bias=True)
)
The model has 4,692,605 trainable parameters


In [34]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=2e-4)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

In [35]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweet  
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.label)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.label)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [36]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweet
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.label)
            acc = binary_accuracy(predictions, batch.label)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [40]:
N_EPOCHS = 10
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.375 | Train Acc: 52.89%
	 Val. Loss: 1.393 |  Val. Acc: 52.01% 

	Train Loss: 1.372 | Train Acc: 53.23%
	 Val. Loss: 1.394 |  Val. Acc: 52.01% 

	Train Loss: 1.368 | Train Acc: 53.78%
	 Val. Loss: 1.395 |  Val. Acc: 51.90% 

	Train Loss: 1.364 | Train Acc: 54.62%
	 Val. Loss: 1.396 |  Val. Acc: 51.45% 

	Train Loss: 1.359 | Train Acc: 55.74%
	 Val. Loss: 1.397 |  Val. Acc: 51.06% 

	Train Loss: 1.353 | Train Acc: 56.64%
	 Val. Loss: 1.398 |  Val. Acc: 50.39% 

	Train Loss: 1.345 | Train Acc: 57.56%
	 Val. Loss: 1.400 |  Val. Acc: 50.22% 

	Train Loss: 1.337 | Train Acc: 58.04%
	 Val. Loss: 1.400 |  Val. Acc: 49.83% 

	Train Loss: 1.326 | Train Acc: 58.97%
	 Val. Loss: 1.402 |  Val. Acc: 49.61% 

	Train Loss: 1.318 | Train Acc: 59.72%
	 Val. Loss: 1.405 |  Val. Acc: 49.05% 



In [38]:
#load weights and tokenizer

path='./saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "very Negative", 1:"Negative", 2:"Neutral", 3: "positive", 4: "very positive"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [39]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

'very Negative'

In [56]:
print("Tweet", "                     ","prediction", "         ", "Actual")

categories = {0: "very Negative", 1:"Negative", 2:"Neutral", 3: "positive", 4: "very positive"}
count = 0
for i in range(1,1000):
  text = ' '.join(vars(valid.examples[i])['tweet']).split(', ')[0]
  if (len (text) > 10) and (classify_tweet(text) == categories [vars(valid.examples[i])['label']]):
    print (text,"  ", classify_tweet(text), "       ", categories [vars(valid.examples[i])['label']] )
    print("\"")
    count= count +1 
    if count >= 10:
      break

    


Tweet                       prediction           Actual
Action mechanical    Neutral         Neutral
"
The story moldy obvious    very Negative         very Negative
"
The movie fails portray literarily talented notorious subject anything much dirty old man    very Negative         very Negative
"
A comprehensive provocative film -- one pushes boundary biography    very Negative         very Negative
"
fear dot com rambling disconnected never builds suspense    very Negative         very Negative
"
Frustratingly    Negative         Negative
"
Director Brian Levant    very Negative         very Negative
"
Purposefully shocking eroticized gore    Neutral         Neutral
"
Though never rises full potential film    Neutral         Neutral
"
soul 's lacking every character movie    very Negative         very Negative
"
